In [20]:
import jsonlines
import os
from pathlib import Path
import pandas as pd
from collections import defaultdict
from utils.get_processed_dataset import get_processed_dataset
from utils.utils import read_jsonl, get_major_entities
from omegaconf import OmegaConf
from configs.config import PRONOUNS_GROUPS, PLURAL_PRONOUNS, dataset_yaml, selected_keys
from configs.config_gen import NAME_TO_PREFIX
from tqdm.auto import tqdm
import hydra
from utils.qa_utils import write_qa_to_jsonl, get_mention_info, add_copelands_count
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import copy

In [ ]:
data_jsonl = "data/qas/data/qas_test.jsonl"
human_data_folder = "data/human_data"
data_test = {}
with jsonlines.open(data_jsonl) as reader:
    for obj in reader:
        data_test[(obj["doc_key"], obj["mention_ind"])] = obj
# print(data_test)

In [14]:
data_test.keys()

dict_keys([('1342_pride_and_prejudice_brat_0', 202), ('2891_howards_end_brat_0', 36), ('155_the_moonstone_brat_0', 244), ('2005_piccadilly_jim_brat_0', 200), ('2891_howards_end_brat_0', 199), ('345_dracula_brat_0', 241), ('24_o_pioneers_brat_0', 49), ('4276_north_and_south_brat_0', 275), ('120_treasure_island_brat_0', 90), ('41286_miss_marjoribanks_brat_0', 312), ('24_o_pioneers_brat_0', 108), ('5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat_0', 101), ('502_desert_gold_brat_0', 46), ('24_o_pioneers_brat_0', 85), ('5230_the_invisible_man_a_grotesque_romance_brat_0', 125), ('217_sons_and_lovers_brat_0', 295), ('969_the_tenant_of_wildfell_hall_brat_0', 280), ('3268_the_mysteries_of_udolpho_brat_0', 33), ('514_little_women_brat_0', 183), ('1342_pride_and_prejudice_brat_0', 312), ('4276_north_and_south_brat_0', 259), ('233_sister_carrie_a_novel_brat_0', 65), ('41286_miss_marjoribanks_brat_0', 149), ('829_gullivers_travels_into_several_remote_nations_of_the_world_brat_0

In [27]:
example_text = data_test[('1342_pride_and_prejudice_brat_0', 202)]["text"]
start = example_text.index("{{")
end = example_text.index("}}")
example_text_cp = copy.deepcopy(example_text)
example_text_cp = example_text_cp.replace("{{", "").replace("}} (#This is the marked mention)", "")
start_mod, end_mod = start - 2, end - 2
# print(example_text_cp[start_mod+1:end_mod])
# print(len(example_text_cp[start_mod+1:end_mod]))

 his wife
9


In [11]:
dataset_configs = OmegaConf.load(dataset_yaml)
mention_info_df_final = pd.DataFrame()
major_entities_all = {}
dataset_proc_all = {}
docs_in_datasets = {}
for dataset_name in ["litbank", "fantasy"]:
    dataset_source = dataset_configs[dataset_name][f"train_file"]
    tsv_addr = dataset_configs[dataset_name][f"tsv"]
    doc_me = dataset_configs[dataset_name][f"train_me"]
    dataset = read_jsonl(dataset_source)
    dataset_proc = get_processed_dataset(dataset, tsv_litbank=tsv_addr)
    dataset_proc_all.update(dataset_proc)
    major_entities = get_major_entities(doc_me)
    major_entities_all.update(major_entities)
    mention_info_df = get_mention_info(dataset_proc, major_entities, tsv_addr=tsv_addr)
    mention_info_df["dataset"] = dataset_name
    mention_info_df_final = pd.concat([mention_info_df_final, mention_info_df])
    docs_in_datasets[dataset_name] = mention_info_df["doc_key"].unique()

/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/thinc/shims/pytorch.py:253: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



599_vanity_fair_brat_0
155_the_moonstone_brat_0
2641_a_room_with_a_view_brat_0
1245_night_and_day_brat_0
2891_howards_end_brat_0
2166_king_solomons_mines_brat_0
3457_the_man_of_the_forest_brat_0
932_the_fall_of_the_house_of_usher_brat_0
2084_the_way_of_all_flesh_brat_0
514_little_women_brat_0
120_treasure_island_brat_0
2005_piccadilly_jim_brat_0
27_far_from_the_madding_crowd_brat_0
62_a_princess_of_mars_brat_0
74_the_adventures_of_tom_sawyer_brat_0
36_the_war_of_the_worlds_brat_0
11_alices_adventures_in_wonderland_brat_0
432_the_ambassadors_brat_0
24_o_pioneers_brat_0
32_herland_brat_0
2775_the_good_soldier_brat_0
5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat_0
9830_the_beautiful_and_damned_brat_0
78_tarzan_of_the_apes_brat_0
1064_the_masque_of_the_red_death_brat_0
209_the_turn_of_the_screw_brat_0
41_the_legend_of_sleepy_hollow_brat_0
1400_great_expectations_brat_0
217_sons_and_lovers_brat_0
3268_the_mysteries_of_udolpho_brat_0
73_the_red_badge_of_courage_an_epis

/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/thinc/shims/pytorch.py:253: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



124_The_Three_Brothers_0
007_The_Good_Bargain_0
077_Clever_Grethel_0
135_The_White_Bride_and_the_Black_One_0
083_Hans_in_Luck_0
044_Godfather_Death_0
016_The_Three_Snake-Leaves_0
049_The_Six_Swans_0
193_The_Drummer_0
157_The_Sparrow_And_His_Four_Children_0
142_Simeli_Mountain_0
103_Sweet_Porridge_0
009_The_Twelve_Brothers_0
211_The_Twelve_Idle_Servants_0
006_Faithful_John_0
112_The_Flail_From_Heaven_0
176_The_Duration_of_Life_0
184_The_Nail_0
060_The_Two_Brothers_0
105_Stories_about_Snakes_0
037_Thumbling_0
153_The_Star-Money_0
113_The_Two_Kings’_Children_0
137_The_Three_Black_Princesses_0
210_The_Hazel-Branch_0
173_The_Bittern_and_the_Hoopoe_0
141_The_Lambkin_and_the_Little_Fish_0
131_Fair_Katrinelje_and_Pif-Paf-Poltrie_0
192_The_Master-Thief_0
090_The_Young_Giant_0
097_The_Water_of_Life_0
159_The_Ditmarsch_Tale_of_Wonders_0
086_The_Fox_and_the_Geese_0
003_Our_Lady’s_Child_0
132_The_Fox_and_the_Horse_0
119_The_Seven_Swabians_0
053_Little_Snow-white_0
188_The_Spindle,_The_Shuttle,_and_

In [ ]:
# selected_dataset = "fantasy"
# selected_test_data = {}
# for doc_key, mention_ind in data_test:
#     if doc_key in docs_in_datasets[selected_dataset]:
#         selected_test_data[(doc_key, mention_ind)] = data_test[(doc_key, mention_ind)]

# sel_test_data_nom = {}
# sel_test_data_pron = {}
# for key in selected_test_data:
#     if selected_test_data[key]["category"] == "NOM":
#         sel_test_data_nom[key] = selected_test_data[key]
#     else:
#         sel_test_data_pron[key] = selected_test_data[key]
# print(len(selected_test_data))        

600


In [ ]:
# ## Set random seed for numpy array
# import numpy as np
# np.random.seed(2)

# human_data_folder = "data/human_data"
# selected_data_documents = set([data_key[0] for data_key in selected_test_data])

# ## Select random 10 documents from the selected dataset
# selected_data_documents = list(selected_data_documents)
# np.random.shuffle(selected_data_documents)
# selected_data_documents = selected_data_documents[:10]

# ## Create the subset of sel_test_data_nom
# human_nom_data = {}
# for key in sel_test_data_nom:
#     if key[0] in selected_data_documents:
#         human_nom_data[key] = sel_test_data_nom[key]

# ## Create the subset of sel_test_data_pron
# human_pron_data = {}
# for key in sel_test_data_pron:
#     if key[0] in selected_data_documents:
#         human_pron_data[key] = sel_test_data_pron[key]

# # print(f"Number of documents in the selected dataset: {len(selected_data_documents)}")
# print(f"Number of NOM mentions in the selected dataset: {len(human_nom_data)}")
# print(f"Number of PRON mentions in the selected dataset: {len(human_pron_data)}")

# ## Select 25 from each and write it to file
# human_nom_data_25 = {}
# human_pron_data_25 = {}
# for key in human_nom_data:
#     if len(human_nom_data_25) < 25:
#         human_nom_data_25[key] = human_nom_data[key]

# for key in human_pron_data:
#     if len(human_pron_data_25) < 25:
#         human_pron_data_25[key] = human_pron_data[key]

# ## Write the data to file
# human_data_folder = "data/human_data"
# os.makedirs(human_data_folder, exist_ok=True)
# with jsonlines.open(f"{human_data_folder}/human_nom_data.jsonl", "w") as writer:
#     for key in human_nom_data_25:
#         writer.write(human_nom_data_25[key])

# with jsonlines.open(f"{human_data_folder}/human_pron_data.jsonl", "w") as writer:
#     for key in human_pron_data_25:
#         writer.write(human_pron_data_25[key])

Number of NOM mentions in the selected dataset: 26
Number of PRON mentions in the selected dataset: 28


In [76]:
JSON_TEMPLATE = {
  "data": {
    "text": "",
    "variants": [],
    "gt": "",
    "doc_key": "",
    "mention_ind": 0,
  }
}

In [ ]:
import json

## Read nom data
human_nom_data = {}
with jsonlines.open(f"{human_data_folder}/human_nom_data.jsonl") as reader:
    for obj in reader:
        human_nom_data[(obj["doc_key"], obj["mention_ind"])] = obj

human_pron_data = {}
with jsonlines.open(f"{human_data_folder}/human_pron_data.jsonl") as reader:
    for obj in reader:
        human_pron_data[(obj["doc_key"], obj["mention_ind"])] = obj

# ## Create the final data
# human_json_folder = "data/human_data/jsons"
# os.makedirs(human_json_folder, exist_ok=True)

# human_data = {}
# for key in human_nom_data:
#     human_data[key] = human_nom_data[key]
# for key in human_pron_data:
#     human_data[key] = human_pron_data[key]

# for key in human_data:
#     ## Fill json template
#     json_obj = copy.deepcopy(JSON_TEMPLATE)
    
#     example_text = human_data[key]["text"]
#     category = human_data[key]["category"]
#     start, end = example_text.index("{{"), example_text.index("}}")
    
#     example_text_cp = copy.deepcopy(example_text)
#     example_text_cp = example_text_cp.replace("{{", "🟢").replace("}} (#This is the marked mention)", "🟢")
#     start_mod, end_mod = start+1, end - 1
    
#     print(example_text_cp[start_mod:end_mod])
#     print(len(example_text_cp[start_mod:end_mod]))
    
#     json_obj["data"]["text"] = example_text_cp
#     json_obj["data"]["variants"] = [{"value": option} for option in human_data[key]["options"]]
#     json_obj["data"]["gt"] = human_data[key]["answer"]
#     json_obj["data"]["doc_key"] = key[0]
#     json_obj["data"]["mention_ind"] = key[1]
#     # json_obj["annotations"][0]["result"][0]["value"]["start"] = start_mod
#     # json_obj["annotations"][0]["result"][0]["value"]["end"] = end_mod
#     # json_obj["annotations"][0]["result"][0]["value"]["text"] = example_text_cp[start_mod:end_mod]

#     with open(f"{human_json_folder}/{key[0]}-{key[1]}-{category}.json", "w") as writer:
#         json.dump(json_obj, writer)

a cupful
8
his bread
9
father
6
the thievish inn-keeper
23
the stall
9
his brother 's table
20
Redskin 's
10
the innkeeper
13
a messenger
11
his mother
10
the youth
9
her palace
10
the third son
13
my name
7
a star on her forehead
22
the water
9
his house
9
a girl he loved
15
our father
10
a loaf of bread
15
his father
10
his pocket
10
a cup which stood near
22
a table which will cover itself
31
an inn which was filled with guests
35
him
3
thy
3
thou
4
he
2
his
3
I
1
I
1
him
3
he
2
thyself
7
I
1
thou
4
I
1
I
1
him
3
he
2
I
1
him
3
me
2
my
2
thou
4
thou
4
me
2
him
3
him
3


In [90]:
## Read file:
model_names = ["mistral_7b_instruct", "llama3_instruct", "gpt4o_mini", "gem_flash", "gpt4o"]
for model_name in model_names:
    file_addr = f"results/qa/{model_name}/desc/output_test.jsonl"
    model_data = {}
    with jsonlines.open(file_addr) as reader:
        for obj in reader:
            model_data[(obj["doc_key"], obj["mention_ind"])] = obj

    num_correct = 0
    for key in human_nom_data:
        model_output = model_data[key]
        answer = model_output["answer"]
        predicted_answer = model_output["predicted_answer"]
        num_correct += (answer == predicted_answer)

    for key in human_pron_data:
        model_output = model_data[key]
        answer = model_output["answer"]
        predicted_answer = model_output["predicted_answer"]
        num_correct += (answer == predicted_answer)

    print(model_name)
    print(f"Number of correct predictions: {num_correct}")
    print(f"Length of data: {len(human_nom_data) + len(human_pron_data)}")
    print(f"Accuracy: {num_correct/(len(human_nom_data) + len(human_pron_data))}")

mistral_7b_instruct
Number of correct predictions: 23
Length of data: 50
Accuracy: 0.46
llama3_instruct
Number of correct predictions: 25
Length of data: 50
Accuracy: 0.5
gpt4o_mini
Number of correct predictions: 31
Length of data: 50
Accuracy: 0.62
gem_flash
Number of correct predictions: 33
Length of data: 50
Accuracy: 0.66
gpt4o
Number of correct predictions: 35
Length of data: 50
Accuracy: 0.7


In [ ]:
# for(key, value) in gpt4o_data.items():
#     print(value.keys())
#     break

dict_keys(['doc_key', 'mention_ind', 'category', 'text', 'prompt', 'options', 'answer', 'output', 'predicted_answer'])


In [89]:
## read trial.csv
annotators = ["skawshik11@gmail.com", "srmsundar93@gmail.com"]
human_trial_data = pd.read_csv(f"{human_data_folder}/m_k_full.csv")

for annotator in annotators:
    human_trial_data_k = human_trial_data[human_trial_data["annotator"] == annotator]

    num_correct = 0
    for index, row in human_trial_data_k.iterrows():
        if row["gt"] == row["choose"]:
            num_correct += 1
    
    print(annotator)
    print(f"Number of correct predictions: {num_correct}")
    print(f"Total number of predictions: {len(human_trial_data_k)}")
    print(f"Accuracy: {num_correct/len(human_trial_data_k)}")

skawshik11@gmail.com
Number of correct predictions: 47
Total number of predictions: 50
Accuracy: 0.94
srmsundar93@gmail.com
Number of correct predictions: 46
Total number of predictions: 50
Accuracy: 0.92
